In [1]:
# Import libraries
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from collections import Counter
import random
import math
from statistics import mode 
import itertools
import numpy as np

# TFIDF
vectorizer = TfidfVectorizer()

In [2]:
def calculate_accuracy_user(model_name):
    correct_pred = 0
    for id_num in ids_test:
        c = Counter(df_test[df_test["id"] == id_num]["predictions"])  
        mode_count = max(c.values())
        mode = {key for key, count in c.items() if count == mode_count}
        pred = next(x for x in df_test[df_test["id"] == id_num]["predictions"] if x in mode)
        #real = mode(df_test[df_test["id"] == id_num]["location"])
        lst  =(list(df_test[df_test["id"] == id_num]["location"]))
        real = max(set(lst), key=lst.count)
        if real == pred:
            correct_pred = correct_pred + 1

    print("Accuracy for {}: {}".format(model_name, correct_pred/len(ids_test)))

In [3]:
df_processed = pd.read_csv("tweet_dataset_processed.csv")

In [9]:
len(df_processed)

45212

# Prediction

In [10]:
# Shuffle all unique ids
df_processed = df_processed.dropna()
ids = list(df_processed["id"].unique())
ids_shuffled = random.sample(ids, len(ids))

In [11]:
# Split ids in train/test 0.9/0.1 approximately
# We should predict multiple times with randomly shuffled data to avoid overfitting
print(len(df_processed["id"].unique()))
ids_train = ids_shuffled[:500]
ids_test = ids_shuffled[500:]

559


In [12]:
# Create train and test
df_train = df_processed[df_processed['id'].isin(ids_train)]
df_test = df_processed[df_processed['id'].isin(ids_test)]

X_train = df_train["tweet"]
X_test = df_test["tweet"]
y_train = df_train["location"]
y_test = df_test["location"]

In [13]:
# Transform tweets with TFIDF vectorizer
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [14]:
# Dummy prediction: Random Forest Classifier
# We are predicting just individual tweets

models = [["Random Forest", RandomForestClassifier()], ["Linear SVC", LinearSVC()], ["Logistic Reg", LogisticRegression()]]

for sel_model in models:
    model_name, model = sel_model
    model.fit(X_train,y_train)
    prediction = model.predict(X_test)
    df_test["predictions"] = prediction
    calculate_accuracy_user(model_name)
     

C:\Users\ignac\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\ignac\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Accuracy for Random Forest: 0.4406779661016949
Accuracy for Linear SVC: 0.576271186440678


C:\Users\ignac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ignac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy for Logistic Reg: 0.5084745762711864
